In [208]:
import PyPDF2
import json

reader = PyPDF2.PdfReader(r"F:\doc\Braunwalds Heart Disease, 12th ed.pdf")


In [209]:
outline = reader.outline

In [199]:
page = reader.pages[0].get_object()

In [206]:
parts = []


def visitor_body(text, cm, tm, fontDict, fontSize):
    y = tm[5]

    # tm [1,0,0,1,tx,ty] 4 tham so dau co the tao quay, skew, scale,...
    # if fontDict and '/FontDescriptor' in fontDict:
    parts.append((tm,fontDict, fontSize,text))


page.extract_text(visitor_text=visitor_body)
# text_body = "".join(parts)

parts[:20]

[([1.0, 0.0, 0.0, 1.0, 0.0, 0.0], None, 12.0, ''),
 ([1.0, 0.0, 0.0, 1.0, 0.0, 0.0], None, 12.0, ''),
 ([1.0, 0.0, 0.0, 1.0, 0.0, 0.0], None, 12.0, ''),
 ([1.0, 0.0, 0.0, -1.0, 104.94509890000002, 29.0],
  {'/Type': '/Font',
   '/Subtype': '/Type0',
   '/BaseFont': '/AAAAAA+ArialMT',
   '/Encoding': '/Identity-H',
   '/DescendantFonts': [IndirectObject(1430, 0, 2648035714432)],
   '/ToUnicode': {'/Filter': '/FlateDecode'}},
  10.6599998,
  '3/2/24, 11:31 AM'),
 ([1.0, 0.0, 0.0, -1.0, 104.94509890000002, 29.0], None, 12.0, ''),
 ([1.0, 0.0, 0.0, 1.0, 0.0, 0.0], None, 12.0, ''),
 ([1.0, 0.0, 0.0, -1.0, 617.1017510999998, 29.0],
  {'/Type': '/Font',
   '/Subtype': '/Type0',
   '/BaseFont': '/AAAAAA+ArialMT',
   '/Encoding': '/Identity-H',
   '/DescendantFonts': [IndirectObject(1430, 0, 2648035714432)],
   '/ToUnicode': {'/Filter': '/FlateDecode'}},
  10.6599998,
  ' Pathophysiology and natural history of mitral stenosis - UpToDate'),
 ([1.0, 0.0, 0.0, -1.0, 617.1017510999998, 29.0], None,

In [ ]:
parts

In [67]:
reader.pages[100].extract_text()

'Impact of Health Care Policy on Quality, Outcomes, and Equity in Cardiovascular Disease65\n6\nlower mortality among patients undergoing PCI in reporting versus \nnon reporting states,  overall outcomes for AMI have been,  if anything,  \nworse in reporting states,  suggesting that selection bias has driven the \napparent improvements in procedural mortality .29\nPublic reporting for hospital processes and outcomes has not been \nassociated with unintended consequences,  but public reporting for \nPCI has been associated with a negative safety signal.  Use of coronary \nangiograms and PCI for AMI are lower in reporting states compared to \n nonreporting states,  and these differences are highest among critically \nill patients,  such as those in cardiogenic shock,  who may benefit most \nfrom the procedure.  This is likely due to risk aversion; the majority of \nsurveyed interventional cardiologists admit to avoiding high risk but \nindicated PCIs due to concern that a bad outcome migh

In [3]:
def parse_outline(reader,outline,doc_name = 'Document'):
    """return nested dict of outline"""
    def sub(outline, prev_nodes:list):
        """return current node and update previous node
        if `o` is a list, use previous node as parent and update its children"""
        if isinstance(outline,PyPDF2.generic.Destination):
            first_page = reader.get_destination_page_number(outline)
            res = {'title':outline.title,
                   'page_range':[first_page,first_page],
                   'children':[]}
            if prev_nodes is not None:
                for node in prev_nodes:
                    node['page_range'][1] = first_page
            return res, [res]
        
        if isinstance(outline, list) and len(prev_nodes) == 1:
            res = []
            last_nodes = [{'page_range':[0,0]}]
            for o in outline:
                cur, nodes = sub(o, last_nodes)
                last_nodes = nodes
                if cur is not None:
                    res.append(cur)
            
            prev_nodes[0]['children'] = res
            for node in nodes:
                prev_nodes.append(node)
            return None, prev_nodes
        
        raise TypeError(str(type(outline))+str(type(prev_nodes)))
    res = [{'title':doc_name,
            'page_range':[0,0],
            'children':[]}]
    sub(outline=outline,prev_nodes=res)
    num_of_pages = len(reader.pages)
    for node in res:
        node['page_range'][1] = num_of_pages - 1
    return res[0]

def flatten_outline(outline:dict, parent_title = None):
    res = [outline]
    if 'children' in outline:
        children = outline.pop('children')
    else:
        children = None

    if isinstance(parent_title,str):
        outline['title'] = parent_title + ' --- ' + outline['title']

    if children is None or len(children) < 1:
        return res
    else:
        for child in children:
            flat_children = flatten_outline(child, outline['title'])
            for flat_child in flat_children:
                res.append(flat_child)
        return res

In [4]:
nested_outline = parse_outline(reader=reader,outline=outline,doc_name='Heart diseases')
flat_outline = flatten_outline(nested_outline,None)

In [6]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http.models import PointStruct,NamedVector
import json

qd_client = QdrantClient("localhost", port=6333)

collection_name = 'outline_collection'
vec_name = 'bge_large'
qd_client.recreate_collection(
    collection_name=collection_name,
    vectors_config={
        # 'gecko':VectorParams(size=768, distance=Distance.DOT),
        vec_name:VectorParams(size=1024, distance=Distance.DOT)}
)

True

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

In [8]:
def index_flat_outline(flat_outline:list, embed_model, collection_name,qd_client):
    vecs = []
    for i,o in enumerate(flat_outline):
        vecs.append(
        PointStruct(id=i+1, vector={'bge_large':embed_model.get_text_embedding(o['title'])},
                    payload=o))
        
        if i%100 == 0:
            operation_info = qd_client.upsert(
                                    collection_name=collection_name,
                                    wait=True,
                                    points=vecs
    )
            vecs = []

            print(operation_info)
    
    if len(vecs) > 0:
        operation_info = qd_client.upsert(
                                    collection_name=collection_name,
                                    wait=True,
                                    points=vecs)
        print(operation_info)


def search_qdrant(query,client,embed_model,collection_name,vec_name,limit=5):
    vec = embed_model.get_text_embedding(query)
    search_result = client.search(
    collection_name=collection_name, query_vector=NamedVector(
       name=vec_name,
       vector=vec), limit=limit
)
    return search_result

In [9]:
len(flat_outline)

4059

In [10]:
index_flat_outline(flat_outline=flat_outline, embed_model=embed_model,collection_name=collection_name,qd_client=qd_client)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>
operation_id=1 status=<UpdateStatus.COMPLETED: 'completed'>
operation_id=2 status=<UpdateStatus.COMPLETED: 'completed'>
operation_id=3 status=<UpdateStatus.COMPLETED: 'completed'>
operation_id=4 status=<UpdateStatus.COMPLETED: 'completed'>
operation_id=5 status=<UpdateStatus.COMPLETED: 'completed'>
operation_id=6 status=<UpdateStatus.COMPLETED: 'completed'>
operation_id=7 status=<UpdateStatus.COMPLETED: 'completed'>
operation_id=8 status=<UpdateStatus.COMPLETED: 'completed'>
operation_id=9 status=<UpdateStatus.COMPLETED: 'completed'>
operation_id=10 status=<UpdateStatus.COMPLETED: 'completed'>
operation_id=11 status=<UpdateStatus.COMPLETED: 'completed'>
operation_id=12 status=<UpdateStatus.COMPLETED: 'completed'>
operation_id=13 status=<UpdateStatus.COMPLETED: 'completed'>
operation_id=14 status=<UpdateStatus.COMPLETED: 'completed'>
operation_id=15 status=<UpdateStatus.COMPLETED: 'completed'>
operation_id=16 status=<UpdateStat

In [234]:
res = search_qdrant('the symptoms of mitral stenosis',client=qd_client,embed_model=embed_model,collection_name=collection_name,vec_name=vec_name,limit=20)

In [235]:
pages = set()
for o in res:
    pages = pages.union(range(*o.payload['page_range']))


In [236]:
content = []
for p in sorted(list(pages)):
    content.append(reader.pages[p].extract_text())

content = '\n'.join(content)

In [237]:
maxx = 0
for c in content.split('\n'):
    if len(c)> maxx:
        maxx = len(c)
maxx

182

In [238]:
qd_client.recreate_collection(
    collection_name='doc_content',
    vectors_config={
        # 'gecko':VectorParams(size=768, distance=Distance.DOT),
        'bge_large':VectorParams(size=1024, distance=Distance.DOT)}
)

True

In [239]:
vecs = []
window_size = 15
step = 10
trunk_content = content.split('\n')
for i in range(window_size//2, len(trunk_content)-window_size//2,step):
    payload = {'content':'\n'.join(trunk_content[i-window_size//2:i-window_size//2+window_size])}
    vecs.append(
    PointStruct(id=i+1, vector={'bge_large':embed_model.get_text_embedding(payload['content'])},
                payload=payload))
    
    if i%100 == 0:
        operation_info = qd_client.upsert(
                                collection_name='doc_content',
                                wait=True,
                                points=vecs
)
        vecs = []

        print(operation_info)

if len(vecs) > 0:
    operation_info = qd_client.upsert(
                                collection_name='doc_content',
                                wait=True,
                                points=vecs)
    print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [240]:
query = """the symptoms of mitral stenosis."""
res = search_qdrant(query=query,client=qd_client,embed_model=embed_model,collection_name='doc_content',vec_name='bge_large',limit=20)

In [241]:
res_content = []
for a in res[:10]:
    res_content.append(a.payload['content'])
res_content

['TABLE 75.1  Stages of Mitral Stenosis (MS)\nSTAGE DEFINITION VALVE ANATOMY VALVE HEMODYNAMICS *HEMODYNAMIC \nCONSEQUENCES SYMPTOMS\nA At risk for MS Mild valve doming during diastole Normal transmitral flow velocity None None\nB Progressive MS Rheumatic valve changes with \ncommissural fusion and diastolic \ndoming of mitral valve leaflets\nPlanimetered MVA >1.5 cm2Increased transmitral flow velocities\nMVA >1.5 cm2\nDiastolic pressure half-  time <150 \nmsecMild to moderate LA \nenlargement\nNormal pulmonary \npressure at restNone\nC Asymptomatic ',
 '\nMitral Stenosis1441.e1\n75\nMitral\nstenosisNon-\nrheumaticLimited treatment options.\nBMV does not work well.\nSurgery is possible but with\nsomewhat less optimal\nresults. New interventional\nmodalities are comingAlso called degenerative MS (DMS). Most common condition\nin the non-rheumatic category & likely to increase significantly\nas the population ages. Difficult to diagnose and treat.\nHighly preventable, easily\ndiagnosed an

In [ ]:
print('\n---\n\n'.join(res_content))

In [176]:
from bs4 import BeautifulSoup
import requests
import json

In [138]:
page = requests.get('https://www.vinmec.com/vi/tin-tuc/thong-tin-suc-khoe/nhi/su-cham-phat-trien-o-tre-em/')
soup = BeautifulSoup(page.content,'html.parser')

In [173]:
a = requests.get('http://localhost:5000/search?q=hep+van+hai+la&n=5')

In [165]:
class WebPageParser():
    def __init__(self, start_tag='h1',
                 exclude_tags=set(['script', 'style', 'noscript']),
                 get_all_text_tags=set(['p', 'li', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])):
        self.start_tag = start_tag
        self.exclude_tags = exclude_tags
        self.get_all_text_tags = get_all_text_tags
        self._nested_structure = None

    def feed(self, doc) -> None:
        self.webpage = doc

    def parse(self, nested: bool = False) -> str:
        """nested is suitable for embedding search in page
        flat structure should to be used in cross page search embedding"""

        page_content = self.webpage.get_doc_content()
        soup = BeautifulSoup(page_content, 'html.parser')
        if not self._nested_structure:
            self._nested_structure = self._get_nested_structure(soup)
        
        if nested:
            return self._nested_structure
        outer_doc = {'tag':'body',
                     'tag_text':'-'.join(node['tag_text'] for node in self._nested_structure),
                     'children':self._nested_structure}
        res = []
        self._get_flat_structure(outer_doc,res)
        return [(o,self.webpage) for o in res]

    def get_flat_outline(self):
        if not self._nested_structure:
            page_content = self.webpage.get_doc_content()
            soup = BeautifulSoup(page_content, 'html.parser')
            self._nested_structure = self._get_nested_structure(soup)
        res = []
        self._get_flat_outline(self._nested_structure, res)
        return res

    def _get_nested_structure(self, soup: BeautifulSoup) -> dict:

        generations = []
        self._parse_recursive_structure(soup.find(self.start_tag), generations)
        res = []
        for node in generations:
            if node['tag'] == self.start_tag:
                self._concat_text(node)
                res.append(node)
        return res

    def _get_text(self, soup: BeautifulSoup) -> str:
        if not soup:
            return ''
        if soup.name in self.get_all_text_tags:
            return soup.text
        return soup.find(string=True, recursive=False)

    def _get_next(self, soup: BeautifulSoup) -> BeautifulSoup:
        if soup.name in self.get_all_text_tags:
            while soup and not soup.find_next_sibling():
                soup = soup.find_parent()
            return soup.find_next_sibling() if soup else None
        return soup.find_next()

    def _concat_text(self, node: dict) -> str:
        children_text = []
        children = node['children']
        new_children = []
        node['tag_text'] = node['text'] if isinstance(node['text'],str) else ''
        for child in children:
            if len(child['children']) == 0 and isinstance(child['text'], str):
                children_text.append(child['text'])
            elif len(child['children']) > 0:
                new_children.append(child)
                self._concat_text(child)
        node['text'] = '\n'.join(children_text)
        node['children'] = new_children

    def _parse_recursive_structure(self, soup: BeautifulSoup, generations: list):
        if not soup or soup.name == 'footer':
            return None

        if soup.name in self.exclude_tags:
            return self._parse_recursive_structure(self._get_next(soup), generations)
        current = {'tag': soup.name,
                   'text': self._get_text(soup), 'children': []}
        if soup.name.startswith('h'):
            while len(generations) > 0 and generations[-1]['tag'] >= soup.name and generations[-1]['tag'] > 'h1':
                generations.pop()
            if len(generations) > 0 and current['tag'] > 'h1':
                generations[-1]['children'].append(current)
            generations.append(current)
            self._parse_recursive_structure(self._get_next(soup), generations)
        else:
            generations[-1]['children'].append(current)
            self._parse_recursive_structure(self._get_next(soup), generations)

    def _get_flat_outline(self, children: list, res: list):
        for child in children:
            res.append(child['tag']+' - '+child['tag_text'])
            if len(child['children']) > 0:
                self._get_flat_outline(child['children'],res)
    
    def _get_flat_structure(self, parent, res: list):
        for child in parent['children']:
            res.append(child['tag']+' - '+parent['tag_text'] +
                       ' - '+child['tag_text'], child['text'])
            if len(child['children']) > 0:
                self._get_flat_structure(child, res)


class WebPage:
    def __init__(self, content) -> None:
        self.content = content

    def get_doc_content(self):
        return self.content


In [166]:
page = requests.get('https://tamanhhospital.vn/hep-van-hai-la/')
soup = BeautifulSoup(page.content,'html.parser')

In [182]:
pages = json.loads(a.content)
outlines = []
for page in pages:
    print(type(page))
    parser = WebPageParser()
    parser.feed(WebPage(page))
    outlines.append(parser.get_flat_outline())

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [190]:
'\n'.join(map(lambda x: x.replace('\n', ' '),outlines[0]))

'h1 - Hẹp van hai lá: Nguyên nhân, dấu hiệu, biến chứng và phòng ngừa\nheader - \nh3 - Mục lục\nh2 - Hẹp van hai lá là gì?\nh2 - Tim hoạt động như thế nào?\nh2 - Nguyên nhân gây hẹp van hai lá\nh2 - Triệu chứng hẹp van 2 lá\nh2 - Các mức độ đánh giá van hai lá\nh2 - Yếu tố nguy cơ\nh2 - Biến chứng của hẹp van hai lá\nh2 - Khi nào cần đi khám bác sĩ?\nh2 - Chẩn đoán hẹp van hai lá như thế nào?\nh2 - Hẹp van tim hai lá có chữa được không?\nh2 - Phương pháp điều trị hẹp van hai lá\nh3 - Xây dựng chế độ sinh hoạt lành mạnh, khoa học\nh2 - Phòng ngừa bệnh hẹp van hai lá\nh3 -           BÀI VIẾT LIÊN QUAN      \nh2 -               LIÊN HỆ VỚI CHÚNG TÔI         \nh2 -                               ĐỐI TÁC BẢO HIỂM                         '

In [194]:
with open('test_outline.txt','w') as f:
    f.write('\n\n--------\n\n'.join(['\n'.join(map(lambda x: x.replace('\n', ' '),o)) for o in outlines]))

In [133]:
def _concat_text(node: dict) -> str:
    concatenated_text = []
    children = node['children']
    new_children = []
    node['tag_text'] = node['text']
    for child in children:
        if len(child['children']) == 0 and isinstance(child['text'], str):
            concatenated_text.append(child['text'])
        elif len(child['children']) > 0:
            new_children.append(child)
            _concat_text(child)
    node['text'] = '\n'.join(concatenated_text)
    node['children'] = new_children

_concat_text(a[0])

In [134]:
a[0]

{'tag': 'h1',
 'text': '\nDS. Lê Thị Kim Thơ \n21/04/2021\n\n\nLời nhắn gửi BÁC SĨ\n\n\n\n\nChia sẻ bài viết\n\n\n\n\n\n PDF\nNội dung chính\n1 Các dịch vụ phục hồi chức năng\n2 Làm thế nào để lập một kế hoạch phục hồi chức năng\n3 Ai cung cấp các dịch vụ phục hồi chức năng?\n4 Tôi sẽ cần dịch vụ phục hồi chức năng trong bao lâu?\n5 Bảo hiểm y tế có thanh toán cho các dịch vụ phục hồi chức năng không?\nBài viết thứ 6 trong 8 bài thuộc chủ đề \nMột số lưu ý khác\n\xa0\nĐiều trị ung thư có thể rất thách thức với cơ thể của bạn. Phục hồi chức năng thể chất giúp cơ thể bạn hồi phục lại sau ung thư, sau điều trị và những thay đổi về thể chất mà bạn trải qua.\nY học phục hồi chức năng bao gồm phòng ngừa, chẩn đoán, điều trị và tối ưu hóa sự hồi phục từ các bệnh nặng hoặc tổn thương nghiêm trọng. Những dịch vụ phục hồi chức năng ung thư có thể bao gồm nhiều chuyên gia chăm sóc sức khỏe, nguồn lực và kỹ thuật điều trị giúp bạn hồi phục lại về thể chất và cảm xúc. Nhà cung cấp dịch vụ chăm sóc 